**Ejercicio 04: Evaluación de un Sistema de Recuperación de Información**

Nombres: Dilan Andrade, Hernan Sanchez, Galo Tarapues

1. Proporcionar un Conjunto de Datos:

In [12]:
import xml.etree.ElementTree as ET
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import euclidean_distances, cosine_similarity
import pandas as pd

# Procesamiento del corpus
tree = ET.parse('03ranking_corpus.xml')
root = tree.getroot()

corpus = []
for document in root.findall('document'):
    doc_id = document.get('id')
    title = document.find('title').text
    keywords = document.find('keywords').text
    corpus.append({
        'id': doc_id,
        'title': title,
        'keywords': keywords.split(', ')
    })

print("Contenido del Corpus:")
for doc in corpus:
    print(f"Documento ID: {doc['id']}, Título: {doc['title']}, Keywords: {doc['keywords']}")

#Consultas
consultas = [
    "beneficios de la telemedicina en salud crónica",
    "impacto de la nutrición en el rendimiento académico",
    "estrategias para mejorar la salud mental en estudiantes universitarios"
]

print("\nConsultas:")
for idx, consulta in enumerate(consultas, start=1):
    print(f"Consulta {idx}: {consulta}")


#Juicios de relevancia

juicios_relevancia = {
    "Consulta 1": ["1", "10"],   # IDs de documentos relevantes para la primera consulta
    "Consulta 2": ["2", "7"],    # IDs de documentos relevantes para la segunda consulta
    "Consulta 3": ["13", "14", "23"]  # IDs de documentos relevantes para la tercera consulta
}

print("\nJuicios de Relevancia Definidos:")
for consulta, documentos_relevantes in juicios_relevancia.items():
    print(f"{consulta}: Documentos relevantes -> {documentos_relevantes}")


Contenido del Corpus:
Documento ID: 1, Título: El aumento de la telemedicina para el tratamiento de condiciones de salud crónicas., Keywords: ['telemedicina', 'salud crónica', 'tratamiento', 'tecnología médica']
Documento ID: 2, Título: Cómo la nutrición balanceada afecta el rendimiento académico y la salud mental en estudiantes., Keywords: ['nutrición', 'rendimiento académico', 'salud mental', 'estudiantes']
Documento ID: 3, Título: Estudio sobre cómo las relaciones de amistad contribuyen al bienestar de los estudiantes en el campus., Keywords: ['amistad', 'bienestar estudiantil', 'campus', 'relaciones sociales']
Documento ID: 4, Título: El rol de las bibliotecas universitarias en el fomento de la investigación académica., Keywords: ['bibliotecas universitarias', 'investigación', 'academia', 'recursos']
Documento ID: 5, Título: Cómo los espacios verdes en los campus universitarios pueden mejorar la concentración y reducir el estrés., Keywords: ['espacios verdes', 'campus universitario

2. Calcular Resultados de Búsqueda:

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import euclidean_distances, cosine_similarity

# Combinar títulos y keywords en un solo texto para cada documento
document_texts = [" ".join([doc['title']] + doc['keywords']) for doc in corpus]

# Vectorizar el corpus y las consultas usando TF-IDF
vectorizer = TfidfVectorizer()
corpus_tfidf = vectorizer.fit_transform(document_texts)
query_tfidf = vectorizer.transform(consultas)

# Calcular la distancia euclidiana
distancia_euclidiana = euclidean_distances(query_tfidf, corpus_tfidf)
print("\nMatriz de Distancia Euclidiana entre Consultas y Documentos:")
print(pd.DataFrame(distancia_euclidiana, index=[f"Consulta {i+1}" for i in range(len(consultas))],
                   columns=[f"Doc {i+1}" for i in range(len(corpus))]))

# Calcular la distancia coseno
distancia_coseno = cosine_similarity(query_tfidf, corpus_tfidf)
print("\nMatriz de Similitud Coseno entre Consultas y Documentos:")
print(pd.DataFrame(distancia_coseno, index=[f"Consulta {i+1}" for i in range(len(consultas))],
                   columns=[f"Doc {i+1}" for i in range(len(corpus))]))

# Ordenar y mostrar los resultados para cada consulta en ambos sistemas
resultados = {}
for idx, consulta in enumerate(consultas):
    print(f"\nConsulta: {consulta}")

    # Resultados para distancia euclidiana (más relevante primero)
    euclidiana_ordenada = distancia_euclidiana[idx].argsort()
    euclidiana_resultados = [corpus[doc_idx]['id'] for doc_idx in euclidiana_ordenada]
    print("\nDocumentos ordenados por distancia euclidiana (más relevante primero):")
    for doc_idx in euclidiana_ordenada:
        print(f"Documento ID: {corpus[doc_idx]['id']}, Distancia Euclidiana: {distancia_euclidiana[idx][doc_idx]:.4f}")

    # Resultados para distancia coseno (más relevante primero)
    coseno_ordenado = distancia_coseno[idx].argsort()[::-1]
    coseno_resultados = [corpus[doc_idx]['id'] for doc_idx in coseno_ordenado]
    print("\nDocumentos ordenados por distancia coseno (más relevante primero):")
    for doc_idx in coseno_ordenado:
        print(f"Documento ID: {corpus[doc_idx]['id']}, Similitud Coseno: {distancia_coseno[idx][doc_idx]:.4f}")

    resultados[f"Consulta {idx + 1}"] = {
        "Distancia Euclidiana": euclidiana_resultados,
        "Distancia Coseno": coseno_resultados
    }



Matriz de Distancia Euclidiana entre Consultas y Documentos:
               Doc 1     Doc 2     Doc 3     Doc 4     Doc 5     Doc 6  \
Consulta 1  0.944752  1.299586  1.386398  1.372879  1.395867  1.363431   
Consulta 2  1.345996  0.836806  1.374316  1.348691  1.385194  1.349048   
Consulta 3  1.291102  1.117378  1.390055  1.392815  1.316638  1.349783   

               Doc 7     Doc 8     Doc 9    Doc 10  ...    Doc 21    Doc 22  \
Consulta 1  1.286788  1.384989  1.372554  1.377949  ...  1.387553  1.385360   
Consulta 2  1.117503  1.372775  1.299199  1.374653  ...  1.265502  1.373302   
Consulta 3  1.062958  1.333769  1.398851  1.349887  ...  1.379821  1.312753   

              Doc 23    Doc 24    Doc 25    Doc 26    Doc 27    Doc 28  \
Consulta 1  1.300964  1.394407  1.385536  1.208790  1.367424  1.385179   
Consulta 2  1.330045  1.392619  1.373987  1.343851  1.354197  1.373587   
Consulta 3  1.253845  1.188927  1.384023  1.339824  1.354873  1.403173   

              Doc 29    Doc

3. Calcular las Métricas de Evaluación:

In [14]:
from sklearn.metrics import ndcg_score

def calcular_metricas(resultados, relevancia, k=5):
    relevantes_encontrados = len([doc_id for doc_id in resultados[:k] if doc_id in relevancia])
    total_relevantes = len(relevancia)

    # Precisión en el top-k (Prec@k)
    precision_at_k = relevantes_encontrados / k if k > 0 else 0

    # Recall
    recall = relevantes_encontrados / total_relevantes if total_relevantes > 0 else 0

    # F1-score
    f1_score = (2 * precision_at_k * recall) / (precision_at_k + recall) if (precision_at_k + recall) > 0 else 0

    # Mean Average Precision (MAP)
    ap_sum = 0
    num_relevantes = 0
    for i, doc_id in enumerate(resultados[:k]):
        if doc_id in relevancia:
            num_relevantes += 1
            ap_sum += num_relevantes / (i + 1)
    map_score = ap_sum / total_relevantes if total_relevantes > 0 else 0

    # nDCG
    relevancias_binarias = [1 if doc_id in relevancia else 0 for doc_id in resultados[:k]]
    ndcg = ndcg_score([relevancias_binarias], [range(k)], k=k)

    return precision_at_k, recall, f1_score, map_score, ndcg

# Calcular e imprimir métricas para cada consulta y sistema
for idx, consulta in enumerate(consultas):
    relevancia = juicios_relevancia[f"Consulta {idx + 1}"]

    print(f"\nMétricas de Evaluación para Consulta {idx + 1} - '{consulta}'")

    # Distancia Euclidiana
    euclidiana_resultados = resultados[f"Consulta {idx + 1}"]["Distancia Euclidiana"]
    prec_at_k, recall, f1, map_score, ndcg = calcular_metricas(euclidiana_resultados, relevancia)
    print(f"\nDistancia Euclidiana:")
    print(f"Prec@k: {prec_at_k:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}, MAP: {map_score:.4f}, nDCG: {ndcg:.4f}")

    # Distancia Coseno
    coseno_resultados = resultados[f"Consulta {idx + 1}"]["Distancia Coseno"]
    prec_at_k, recall, f1, map_score, ndcg = calcular_metricas(coseno_resultados, relevancia)
    print(f"\nDistancia Coseno:")
    print(f"Prec@k: {prec_at_k:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}, MAP: {map_score:.4f}, nDCG: {ndcg:.4f}")



Métricas de Evaluación para Consulta 1 - 'beneficios de la telemedicina en salud crónica'

Distancia Euclidiana:
Prec@k: 0.2000, Recall: 0.5000, F1-score: 0.2857, MAP: 0.5000, nDCG: 0.3869

Distancia Coseno:
Prec@k: 0.2000, Recall: 0.5000, F1-score: 0.2857, MAP: 0.5000, nDCG: 0.3869

Métricas de Evaluación para Consulta 2 - 'impacto de la nutrición en el rendimiento académico'

Distancia Euclidiana:
Prec@k: 0.4000, Recall: 1.0000, F1-score: 0.5714, MAP: 1.0000, nDCG: 0.5013

Distancia Coseno:
Prec@k: 0.4000, Recall: 1.0000, F1-score: 0.5714, MAP: 1.0000, nDCG: 0.5013

Métricas de Evaluación para Consulta 3 - 'estrategias para mejorar la salud mental en estudiantes universitarios'

Distancia Euclidiana:
Prec@k: 0.4000, Recall: 0.6667, F1-score: 0.5000, MAP: 0.5556, nDCG: 0.5438

Distancia Coseno:
Prec@k: 0.4000, Recall: 0.6667, F1-score: 0.5000, MAP: 0.5556, nDCG: 0.5438


4. Análisis y Comparación:

In [15]:
print("\nAnálisis y Comparación de los Sistemas de Recuperación:")

for idx, consulta in enumerate(consultas):
    relevancia = juicios_relevancia[f"Consulta {idx + 1}"]

    print(f"\nAnálisis para Consulta {idx + 1} - '{consulta}'")

    # Evaluación de cada sistema con base en las métricas calculadas
    euclidiana_resultados = resultados[f"Consulta {idx + 1}"]["Distancia Euclidiana"]
    coseno_resultados = resultados[f"Consulta {idx + 1}"]["Distancia Coseno"]

    # Cálculo de métricas de evaluación
    euclidiana_metrica = calcular_metricas(euclidiana_resultados, relevancia)
    coseno_metrica = calcular_metricas(coseno_resultados, relevancia)

    # Comparar las métricas entre los dos sistemas
    if euclidiana_metrica[2] > coseno_metrica[2]:  # Comparación basada en F1-score
        print("Distancia Euclidiana es más efectiva en esta consulta.")
    elif coseno_metrica[2] > euclidiana_metrica[2]:
        print("Distancia Coseno es más efectiva en esta consulta.")
    else:
        print("Ambos sistemas muestran un desempeño similar.")



Análisis y Comparación de los Sistemas de Recuperación:

Análisis para Consulta 1 - 'beneficios de la telemedicina en salud crónica'
Ambos sistemas muestran un desempeño similar.

Análisis para Consulta 2 - 'impacto de la nutrición en el rendimiento académico'
Ambos sistemas muestran un desempeño similar.

Análisis para Consulta 3 - 'estrategias para mejorar la salud mental en estudiantes universitarios'
Ambos sistemas muestran un desempeño similar.
